In [ ]:
import math
import pandas as pd
import numpy as np
from collections import defaultdict

def loadData(nbChunks = 0):
    asinToIndex, userToIndex = {}, {}
    asin_index, user_index = 0, 0
    asins, users = [], []
    rate_to_user_book = defaultdict(lambda: [])
    chunks = pd.read_json(
        '../Dataset/Books.json', lines=True, chunksize=500000,
        typ="frame", orient="records", dtype={"asin": str, "overall": int, "unixReviewTime": int, "reviewerID": str}
    )
    for chunk in chunks:
        for _, item in chunk.iterrows():
            asin = item["asin"]
            rate = item["overall"]
            user = item["reviewerID"]
            if (asin not in asinToIndex):
                asinToIndex[asin] = asin_index
                asins.append(asin)
                asin_index += 1
            if (user not in userToIndex):
                userToIndex[user] = user_index
                users.append(user)
                user_index += 1
            row = userToIndex[user]
            col = asinToIndex[asin]
            rate_to_user_book[rate-1].append((row, col))
        nbChunks -= 1
        if (nbChunks == 0): break
    Fr1Matrix = [[None]*asin_index for _ in range(user_index)]
    for rate, (i, j) in rate_to_user_book.items():
        Fr1Matrix[i][j] = rate
    Fr1Matrix = np.array(Fr1Matrix)
    nbGivedRatings, nbReceivedRatings = np.sum(Fr1Matrix, axis=1), np.sum(Fr1Matrix, axis=0)
    rowNames, colNames = users + ["TotalBooksReceivedRatings"], asins + ["TotalUsersGivedRatings"]
    nbReceivedRatings.append(np.sum(nbGivedRatings))
    Fr1dataFrame = np.concatenate(np.concatenate((Fr1Matrix, nbGivedRatings), axis=1), nbReceivedRatings)
    Fr1DataFRame = pd.DataFrame(data=Fr1DataFRame, index=rowNames, colums=colNames)
    return Fr1DataFRame

Fr1 = loadData()
display(Fr1)

: 